In [1]:
# get the previous day's fantasy results from FantasyData
import requests
import pandas as pd

# get yesterday's date, convert to string
from datetime import date, timedelta
date = date.today() - timedelta(1)
datestr = date.strftime('%m-%d-%Y')

import requests
session_requests = requests.session()

headers = {
    'authority': 'fantasydata.com',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.9',
    'accept-encoding': 'gzip, deflate',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
}

data = {
    'sort': 'Name-asc',
    'pageSize': '300',
    'group': '',
    'filter': '',
    'filters.scope': '2',
    'filters.subscope': '',
    'filters.season': '2023',
    'filters.seasontype': '1',
    'filters.team': '',
    'filters.conference': '1',
    'filters.position': '',
    'filters.searchtext': '',
    'filters.scoringsystem': '',
    'filters.exportType': '',
    'filters.date': datestr,
    'filters.dfsoperator': '',
    'filters.dfsslateid': '',
    'filters.dfsslategameid': '',
    'filters.dfsrosterslot': '',
    'filters.showfavs': '',
    'filters.teamkey': '',
    'filters.oddsstate': '',
    'filters.showall': '',
}

resp = session_requests.get('https://fantasydata.com/NBA_FantasyStats/FantasyStats_Read', headers=headers, data=data)
print(resp)
data = resp.json() #Grab Data In JSON format
df = pd.json_normalize(data, record_path=['Data']) #Transform Json to dataframe

<Response [200]>


In [2]:
df

,StatID,TeamID,PlayerID,Name,Team,Position,Games,Started,Season,Points,...,Game.AwayTeamScore,Game.Quarter,Game.Status,Game.IsOver,Game.ShouldHaveStarted,Game.TimeRemainingMinutes,Game.TimeRemainingSeconds,Game.DateTime,Game.TimeRemaining,Game.QuarterDisplay
0,1063093,25,20002725,A.J. Lawson,DAL,SG,0,0,2023,0.0,...,105,None,Final,True,True,None,None,/Date(1675128600000)/,,F
1,1063150,3,20002005,Aaron Holiday,ATL,PG,1,1,2023,3.0,...,125,None,Final,True,True,None,None,/Date(1675134000000)/,,F
2,1063035,18,20002714,Aaron Wiggins,OKC,SG,1,1,2023,3.0,...,128,None,Final,True,True,None,None,/Date(1675126800000)/,,F
3,1062935,5,20002311,Admiral Schofield,ORL,PF,0,0,2023,0.0,...,119,None,Final,True,True,None,None,/Date(1675123200000)/,,F
4,1063149,3,20002876,AJ Griffin,ATL,SF,1,0,2023,13.0,...,125,None,Final,True,True,None,None,/Date(1675134000000)/,,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,1062993,16,20002895,Wendell Moore Jr.,MIN,SG,0,0,2023,0.0,...,118,None,F/OT,True,True,None,None,/Date(1675126800000)/,,F/OT
260,1062969,27,20002080,Wenyen Gabriel,LAL,PF,1,0,2023,11.0,...,104,None,Final,True,True,None,None,/Date(1675125000000)/,,F
261,1063046,1,20000887,Will Barton,WAS,SG,0,0,2023,0.0,...,127,None,Final,True,True,None,None,/Date(1675126800000)/,,F
262,1062953,8,20002058,Yuta Watanabe,BKN,PF,1,0,2023,12.0,...,104,None,Final,True,True,None,None,/Date(1675125000000)/,,F


In [3]:
# create dataframe with yesterday's results
results_df = df[['Name', 'FantasyPointsFanDuel']]

In [4]:
# read in model predictions

# change formatting of date string to read in predictions
date = date.today() - timedelta(1)
datestr = date.strftime('%Y%m%d')

import os

working_directory = 'D:/machine_learning/DFS/NBA/NBA_moredata'
os.chdir(working_directory)
pred_dir = 'Predictions/'

# read in model predictions
g_preds = pd.read_csv(pred_dir + 'G_Predictions_'+str(datestr)+'.csv', index_col = 0)
f_preds = pd.read_csv(pred_dir + 'F_Predictions_'+str(datestr)+'.csv', index_col = 0)
c_preds = pd.read_csv(pred_dir + 'C_Predictions_'+str(datestr)+'.csv', index_col = 0)

# concat predictions to a single dataframe
all_preds = pd.concat([g_preds, f_preds, c_preds]).reset_index(drop = True)

# fix names with accent marks/suffixes
import unidecode

for i in range(len(all_preds)):
    # accent mark fix
    all_preds.iloc[i, 2] = unidecode.unidecode(all_preds.iloc[i, 2])
    # drop Jrs.
    if all_preds.iloc[i, 2][-3:] == 'Jr.' or all_preds.iloc[i, 2][-2:] == 'IV':
        all_preds.iloc[i, 2] = all_preds.iloc[i, 2][:-3]

In [5]:
# drop lower prediction for players whose performance was predicted with multiple models
all_preds = all_preds.sort_values('Prediction', ascending = False).drop_duplicates(subset = 'Player')

# rename Player column to Name
all_preds = all_preds.rename({'Player':'Name'}, axis = 1)

In [6]:
results_df = results_df.merge(all_preds, on = 'Name')

In [7]:
from sklearn.metrics import mean_absolute_error
print("{:.2f}".format(mean_absolute_error(results_df['FantasyPointsFanDuel'], results_df['Prediction'])))

10.78


In [11]:
print("{:.2f}".format(mean_absolute_error(results_df['FantasyPointsFanDuel'], results_df['Prediction_RF'])))

10.59


In [12]:
print("{:.2f}".format(mean_absolute_error(results_df['FantasyPointsFanDuel'], results_df['Prediction_LGBM'])))

11.15


In [13]:
results_df.sort_values('FantasyPointsFanDuel', ascending = False).head(n=50)

,Name,FantasyPointsFanDuel,Season,Date,Team,Opp,Prediction,PredictRank,Prediction_RF,Prediction_LGBM
157,Luka Doncic,72.1,2022,20230130,DAL,DET,49.267504,2.0,50.4003,48.134708
58,Dejounte Murray,66.1,2022,20230130,ATL,POR,38.996468,11.0,38.3319,39.661037
212,Stephen Curry,64.6,2022,20230130,GSW,OKC,45.060900,7.0,45.2845,44.837300
148,Kristaps Porzingis,60.3,2022,20230130,WAS,SAS,39.116742,5.0,41.0717,37.161784
112,Joel Embiid,56.7,2022,20230130,PHI,ORL,54.452644,1.0,52.6521,56.253187
12,Anthony Edwards,56.1,2022,20230130,MIN,SAC,43.629486,9.0,44.2996,42.959372
45,Damian Lillard,53.4,2022,20230130,POR,ATL,46.208509,6.0,48.6855,43.731519
170,Mikal Bridges,51.6,2022,20230130,PHO,TOR,32.411417,21.0,32.2431,32.579734
54,De'Aaron Fox,51.0,2022,20230130,SAC,MIN,38.579055,12.0,40.2614,36.896710
151,Kyrie Irving,50.4,2022,20230130,BRK,LAL,44.456409,8.0,43.4581,45.454717


In [14]:
results_df.to_csv('predictions/pred_assessment/pred_vs_actual_' + datestr + '.csv')